In [1]:
from openhexa.sdk import workspace
import pandas as pd
import numpy as np
import json
from utils import *
import pickle


In [2]:
files_path = "workspace"
path = f"{files_path}/PBF burundi extraction/data"
path_metadata = f"{files_path}/PBF burundi extraction/metadata"
path_cds = f"{path}/cds"
path_hop = f"{path}/hopitaux"

387

In [9]:
def prepare_quantite_for_VBR(quant):
    df = pd.pivot_table(
        quant,
        index=["pe", "ou", "service"],
        columns=["de"],
        values="value",
        aggfunc=np.sum,
        fill_value=pd.NA,
    ).reset_index()
    df.rename(columns={"pe": "month","service":"dx_name","declaree_indiv":"dec", "verifiee_indiv":"ver","validee_indiv":"val"}, inplace=True)
    df["gain_verif"] = (df["dec"] - df["val"]) * df["tarif"]
    df["subside_sans_verification"] = df["dec"] * df["tarif"]
    df["subside_avec_verification"] = df["val"] * df["tarif"]
    df["quarter"] = df["month"].map(month_to_quarter)
    df = calcul_ecarts(df)
    return df

def prepare_qualite_for_VBR(qual):
    qual = qual[qual.de == "score_pma"]
    qual = pd.pivot_table(
        qual,
        index=["pe", "ou", "service"],
        columns=["de"],
        values="value",
        aggfunc=np.sum,
        fill_value=pd.NA,
    ).reset_index()
    
    qual.rename(columns={"pe": "quarter","service":"DE_name_officiel"},inplace=True)
    qual["month"] = qual["quarter"].map(quarter_to_months)
    
    return qual

def read_data(path_ou, data_type):
    return pd.read_csv(f"{path_ou}/{data_type}.csv")


def metadata_ou():
    with open(f"{path_metadata}/orgunits.json") as fp:
        ous = json.load(fp)
    return ous


In [ ]:
q_cs = read_data(path_cds, "quantite")
quantite_cs = prepare_quantite_for_VBR(q_cs)

ous = metadata_ou()


C:\Users\Jean-PhilippeHubinon\AppData\Local\Temp\ipykernel_62060\665572664.py:2: FutureWarning: The provided callable <function sum at 0x0000020D27A0F6A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df = pd.pivot_table(


In [ ]:

    def add_parents(df, parents):
        filtered_parents = {key: parents[key] for key in df["ou"] if key in parents}
        # Transform the `parents` dictionary into a DataFrame
        parents_df = pd.DataFrame.from_dict(filtered_parents, orient="index").reset_index()

        # Rename the index column to match the "ou" column
        parents_df.rename(columns={"index": "ou", "level_2_id":"level_2_uid",
                        "level_3_id":
                        "level_3_uid",
                        "level_4_id":
                        "level_4_uid",
                        "level_5_id":
                        "level_5_uid","name":"level_5_name"}, inplace=True)

        # Join the DataFrame with the parents DataFrame on the "ou" column
        result_df = df.merge(parents_df, on="ou", how="left")
        return result_df


    


In [33]:
non_null_dec = quantite_cs[~quantite_cs.dec.isnull()]
non_null = non_null_dec[~non_null_dec.val.isnull()]

by_dx_month = non_null.groupby(["ou","dx_name","month"]).agg({'gain_verif':'sum', 'taux_validation':'mean'}).reset_index()
by_month = by_dx_month.groupby(["dx_name"]).agg({'gain_verif':'mean','taux_validation':'mean', 'ou': 'nunique'})
by_month

de,gain_verif,taux_validation,ou
dx_name,,,
Accouchement dystocique au CDS,264.534884,0.996858,187
Accouchement eutocique,10631.888936,0.982821,578
Cas des IST traitées,466.787518,0.942396,613
Consultation postnatale,40345.608200,0.640317,538
Consultation prénatale recentrée,7117.067420,0.840231,282
Depistage et PEC malnutrition chez les moins de 5 ans,4480.750445,0.896814,533
Dépistage des cas TBC positifs par mois,249.950495,0.995416,131
Enfants complétement vaccinés (VAR2),9771.146784,0.967062,616
FP: Implants et DIU,2443.481350,0.986840,541


In [36]:
avg_gain_verif_by_ou = (
    by_dx_month.groupby(["ou", "month"], as_index=False)
    .agg({"gain_verif": "sum", "taux_validation": "mean"})
)
avg_gain_verif_by_ou.describe()


de,month,gain_verif,taux_validation
count,10550.000000,1.055000e+04,10550.000000
mean,202311.454976,1.425283e+05,0.876625
std,51.094852,4.044245e+05,0.064228
min,202211.000000,-1.651100e+05,0.019231
25%,202303.000000,3.866250e+04,0.837256
50%,202307.000000,8.320500e+04,0.882857
75%,202311.000000,1.720238e+05,0.923440
max,202403.000000,3.664543e+07,1.000000


In [17]:
qual_cs = read_data(path_cds, "qualite")
qual_cs = prepare_qualite_for_VBR(qual_cs)


C:\Users\Jean-PhilippeHubinon\AppData\Local\Temp\ipykernel_62060\665572664.py:20: FutureWarning: The provided callable <function sum at 0x0000020D27A0F6A0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  qual = pd.pivot_table(


In [23]:
#vbr_cs_enriched = add_parents(vbr_cs, ous)

def save_simulation_environment(name, ous, quant, qual, data_path, is_pca):
    regions = []
    orgunits = quant["ou"].unique()
    quant = add_parents(quant, ous)
    qual = add_parents(qual, ous)
    print(quant.sample(1))
    qualite_indicators = sorted(qual.DE_name_officiel.unique())
    group = Group_Orgunits("all_regions", [])
    nb_tot = 0
    for ou in orgunits:
        if len(quant[(quant.level_5_uid == ou)]) >0:
            group.add_ou(
                Orgunit(
                    ou,
                    quant[(quant.level_5_uid == ou)],
                    qual[(qual.level_5_uid == ou)],
                    qualite_indicators,
                    is_pca,)
            )
        nb_tot+=1
    print(f"number of orgunits= {nb_tot}")
    regions.append(group)
    with open(f'{data_path}/initialization_simulation/{name}.pickle', 'wb') as file:
    # Serialize and save the object to the file
        pickle.dump(regions, file)
    print(f"Fichier d'initialisation sauvé: {data_path}/initialization_simulation/{name}.pickle")
    return regions

In [24]:
regions = save_simulation_environment("burundi", ous, quantite_cs, qual_cs, path, False)

        month           ou                                   dx_name  dec  \
38927  202212  x3etAgy5nse  Nouvelle Consultation Curative (>=5 ans)  NaN   

       tarif  val  ver  gain_verif  subside_sans_verification  \
38927  290.0  NaN  NaN         NaN                        NaN   

       subside_avec_verification  ... level_5_name  level_5_uid   level_1_id  \
38927                        NaN  ...  CDS Kiyenzi  x3etAgy5nse  rdX5nU5lrcx   

       level_2_uid  level_3_uid  level_4_uid level_1_name     level_2_name  \
38927  LBhx9OYcHs8  MhfZRdfVtWX  TgcqlP7t2IM      Burundi  Bujumbura Rural   

      level_3_name level_4_name  
38927    DS Kabezi   AR Kiyenzi  

[1 rows x 25 columns]
number of orgunits= 907
Fichier d'initialisation sauvé: workspace/PBF burundi extraction/data/initialization_simulation/burundi.pickle


In [ ]:

def run_simulation(
    regions,
    frequence,
    start,
    end,
    prix_verif,
    seuil_gain_verif_median,
    seuil_max_bas_risk,
    seuil_max_moyen_risk,
    window,
    nb_period_verif,
    proportion_selection_bas_risque,
    proportion_selection_moyen_risque,
    proportion_selection_haut_risque,
    paym_method_NF,
    use_quality_for_risk,
):
    data_path = "/home/hexa/workspace/PBF PMNS extraction/data/"
    for month in [
        int(str(m)) for m in get_date_series(str(start), str(end), frequence)
    ]:
        if frequence == "trimestre" and month % 100 % 3 != 0:
            continue
        current_run.log_info(f"Simule la vérification pour {month}")
        file_verif = f"{data_path}Selections_Verif/FREQ:{frequence}-GAIN_VERIF_MEDIAN_MAX:{seuil_gain_verif_median}-MIN_NB_TRIM_OBS:{window}-MIN_NB_TRIM_WITH_VERIF:{nb_period_verif}-p_low:{proportion_selection_bas_risque}-p_mod:{proportion_selection_moyen_risque}-p_high:{proportion_selection_haut_risque}-cout_verif:{prix_verif}-seuil_max_moyen_risk:{seuil_max_moyen_risk}-seuil_max_bas_risk:{seuil_max_bas_risk}-Paiement:{paym_method_NF}-Quality_risk:{use_quality_for_risk}"
        file_results = f"{data_path}result_simulation/FREQ:{frequence}-GAIN_VERIF_MEDIAN_MAX:{seuil_gain_verif_median}-MIN_NB_TRIM_OBS:{window}-MIN_NB_TRIM_WITH_VERIF:{nb_period_verif}-p_low:{proportion_selection_bas_risque}-p_mod:{proportion_selection_moyen_risque}-p_high:{proportion_selection_haut_risque}-cout_verif:{prix_verif}-seuil_max_moyen_risk:{seuil_max_moyen_risk}-seuil_max_bas_risk:{seuil_max_bas_risk}-Paiement:{paym_method_NF}-Quality_risk:{use_quality_for_risk}.csv"
        simulate(
            regions,
            file_results,
            file_verif,
            frequence,
            month,
            prix_verif,
            seuil_gain_verif_median,
            seuil_max_bas_risk,
            seuil_max_moyen_risk,
            window,
            nb_period_verif,
            proportion_selection_bas_risque,
            proportion_selection_moyen_risque,
            proportion_selection_haut_risque,
            paym_method_NF,
            use_quality_for_risk,
        )
    return file_results


de,month,level_5_uid,dx_name,dec,tarif,val,ver,gain_verif,Quarter,ecart_dec_ver,ecart_ver_val,taux_validation,ecart_dec_val
0,202211,A0j1Mcpr5Q9,Accouchement dystocique au CDS,NaN,26000.0,NaN,NaN,NaN,2022Q4,NaN,NaN,1.0,NaN
1,202211,A0j1Mcpr5Q9,Accouchement eutocique,35.0,26000.0,35.0,35.0,0.0,2022Q4,0.000000,0.0,1.0,0.000000
2,202211,A0j1Mcpr5Q9,Cas des IST traitées,22.0,570.0,23.0,23.0,-570.0,2022Q4,0.043478,0.0,1.0,0.017391
3,202211,A0j1Mcpr5Q9,Consultation postnatale,12.0,1560.0,NaN,12.0,NaN,2022Q4,0.000000,NaN,1.0,NaN
4,202211,A0j1Mcpr5Q9,Consultation prénatale recentrée,8.0,1560.0,NaN,8.0,NaN,2022Q4,0.000000,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286714,202403,za3HSBjLgi9,Nouvelle Consultations Curative Femme Enceinte,173.0,4000.0,173.0,173.0,0.0,2024Q1,0.000000,0.0,1.0,0.000000
286715,202403,za3HSBjLgi9,Petite chirurgie < 5 ans,43.0,3500.0,NaN,NaN,NaN,2024Q1,NaN,NaN,1.0,NaN
286716,202403,za3HSBjLgi9,Prise en charge des victimes des violences sex...,NaN,40250.0,NaN,NaN,NaN,2024Q1,NaN,NaN,1.0,NaN
286717,202403,za3HSBjLgi9,Prise en charge du nouveau né d'une femme VIH+,NaN,15300.0,NaN,NaN,NaN,2024Q1,NaN,NaN,1.0,NaN
